In [3]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from pandas.io.json import json_normalize


## Using the Notebook to build the code to scrape the following Wikipedia page in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [11]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

res = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') 
soup = BeautifulSoup(res.content,'lxml') # get the content from the website
#print(soup.prettify())

table = soup.find_all('table',{'class':'wikitable sortable'})[0] # find the table on the website
table_rows = table.find_all('tr') # extract the rows of the column from the table

values = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        values.append(row) # extra the rows and columns and put the text in an empty list

df = pd.DataFrame(values, columns=["Postcode", "Borough", "Neighbourhood"]) # create the dataframe from the list

df = df.loc[df['Borough'] != "Not assigned"] # drop columns where borough has values 'not assigned'
df = df.reset_index() # reset the index to start from 0 again
df = df.drop(columns = 'index') # drop the column with the old index
df = df.groupby(['Postcode', 'Borough'],as_index=False)['Neighbourhood'].agg(','.join) # collapse rows with the same postcode and separate them with a comma

df.Neighbourhood.replace('Not assigned',df.Borough, inplace=True) # Replace empty neighbourhood values with the value in the borough column
# df.to_csv('./Desktop/projects/CourseraCapstone/postcodes.csv') # save it as csv file so it can be used in the next exercise
df

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [ ]:
df.shape # the dataframe has 103 rows and 3 columns